#### Clarusway Python

* [Instructor Landing Page](landing_page.ipynb)
* <a href="https://colab.research.google.com/github/4dsolutions/clarusway_data_analysis/blob/main/basic_python/sandbox_week_07.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
* [![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/4dsolutions/clarusway_data_analysis/blob/main/basic_python/sandbox_week_07.ipynb)

# Sandbox 7: Completing Python


In [1]:
class PantsOnFire(Exception):
    pass

def test_me():
    try:
        print("I am testing you now")
        1/0
    except:
        print("bad!")
        raise PantsOnFire("Don't worry though!")

try:
    test_me()
except PantsOnFire as e:
    print(e)
else:
    print("Never got here")
finally:
    print("That was fun")

I am testing you now
bad!
Don't worry though!
That was fun


In [2]:
def compose(f, g):
    def h(x):
        return f(g(x))
    return h

def m(s):
    return s + "s"

def n(s):
    return s.upper()

funk1 = compose(m, n)
funk2 = compose(n, m)

In [3]:
funk1("Hello")

'HELLOs'

In [4]:
funk2("Hello")

'HELLOS'

In [5]:
h = compose(lambda x: 2*x, lambda y: y+2)

In [6]:
h(10)

24

In [7]:
(lambda x: 2*x)((lambda y: y+2)(10))

24

In [8]:
h = compose(lambda y: y+2, lambda x: 2*x)

In [9]:
h(10)

22

In [10]:
(lambda y: y+2)((lambda x: 2*x)(10))

22

In [11]:
class Compose:

    def __init__(self, f):
        self.f = f
    def __call__(self, x):
        return self.f(x)
    def __mul__(self, g):
        return Compose(lambda x: g(self.f(x))) # creates new object with lambda

funk1 = Compose(funk1) # funk1: upper then add s
funk2 = Compose(funk2) # funk2: add s then upper

funk3 = funk1 * funk2  # add s then upper then upper and add s
funk4 = funk2 * funk1  # upper then add s then add s then upper

In [12]:
funk3("bravo")

'BRAVOSS'

In [13]:
funk4("bravo")

'BRAVOSs'

Exercise: use Compose class with simple f, g of your own making. Show (f * g)(x) and (g * f)(x)

## Bonus Topic: Decorator Syntax

Take a look at this syntax:

```python
    funk1 = Compose(funk1) # funk1: upper then add s
    funk2 = Compose(funk2) # funk2: add s then upper
```

The input and output have the same name, but a transformation is happening. funk1 is going in as a function but coming out as a Compose type instance.

When you want to eat a callable and return a transformed version with the same name, Python offers "decorator syntax" (@).

In [14]:
@Compose
def m(x): return x + 2

@Compose
def n(x): return 2 * x

In [15]:
(m * n)(10)

24

In [16]:
(n * m)(10)

22

## Bonus Topics: Properties, Class and Static Methods

Now that you have seen decorator syntax, we can make sense of some additional syntax.

The property decorator allows a method to disguise itself as simply an attribute. 

Different methods apply for getting (shown) and setting (not shown) a property.

Think of attributes as private in the sense that you wish to protect them with code.

In [17]:
from datetime import date

class Animal:

    def __init__(self, dob):
        self.dob = dob
        self.stomach = []

    @property
    def age(self):
        return (date.today() - self.dob).days

    def eat(self, food):
        self.stomach.append(food)

    def __repr__(self):
        return "Animal born {}".format(self.dob)

In [18]:
dog = Animal(date(1970, 3, 3))

In [19]:
dog

Animal born 1970-03-03

In [20]:
dog.dob.isoformat()

'1970-03-03'

In [21]:
dog.age

19994

The classmethod decorator allows a method accept the class itself in place of self, such that the method is about doing something to the class, not the instances directly.

In [22]:
import random 

class Dog(Animal):
    
    tricks = ["roll over", "play dead", "fetch stick"]
    
    def __init__(self, name, dob):
        self.name = name
        super().__init__(dob) # pass dob to superclass __init__

    def do_trick(self):
        return random.choice(self.tricks) # self.tricks checks self.__dict__ then class.__dict__
        
    @classmethod
    def add_trick(klass, trick):  # add a trick to the class-level list named tricks
        klass.tricks.append(trick)

In [23]:
rover = Dog("Rover", date(2000, 1, 10))

In [24]:
rover.age

9089

In [25]:
rover.do_trick()

'roll over'

In [26]:
rover.do_trick()

'fetch stick'

In [27]:
rover.add_trick("beg")

In [28]:
Dog.tricks

['roll over', 'play dead', 'fetch stick', 'beg']